In [1]:
!pip install doclayout-yolo==0.0.3 gradio==5.1.0 gradio-client==1.4.0
!pip install -q ultralytics pymupdf opencv-python pillow
!pip install -q pytesseract
!apt-get install -y tesseract-ocr
!apt-get install -y tesseract-ocr-vie

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  tesseract-ocr-vie
0 upgraded, 1 newly installed, 0 to remove and 1 not upgraded.
Need to get 417 kB of archives.
After this operation, 546 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-vie all 1:4.00~git30-7274cfa-1.1 [417 kB]
Fetched 417 kB in 0s (2,357 kB/s)
Selecting previously unselected package tesseract-ocr-vie.
(Reading database ... 117528 files and directories currently installed.)
Preparing to 

In [2]:
from huggingface_hub import snapshot_download
import os
import fitz
import cv2
import torch
import numpy as np
from PIL import Image
from doclayout_yolo import YOLOv10
import pytesseract
import re
import requests
from urllib.parse import urlparse

In [3]:
# == download weights ==
model_dir = snapshot_download('juliozhao/DocLayout-YOLO-DocStructBench-imgsz1280-2501', local_dir='./models/DocLayout-YOLO-DocStructBench-imgsz1280-2501')
# == select device ==
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

.gitattributes: 0.00B [00:00, ?B/s]

doclayout_yolo_docstructbench_imgsz1280_(…):   0%|          | 0.00/39.8M [00:00<?, ?B/s]

In [4]:
MODEL_PATH = "./models/DocLayout-YOLO-DocStructBench-imgsz1280-2501/doclayout_yolo_docstructbench_imgsz1280_2501.pt"

model = YOLOv10(MODEL_PATH)
model.to(DEVICE)

YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, trac

In [5]:
def download_pdf_from_url(url, save_dir="input", chunk_size=8192):
    os.makedirs(save_dir, exist_ok=True)
    filename = os.path.basename(urlparse(url).path)
    if not filename.endswith(".pdf"):
        filename = "document.pdf"

    save_path = os.path.join(save_dir, filename)
    if os.path.exists(save_path):
        print(f"PDF existed: {save_path}")
        return save_path

    print(f"Downloading PDF to {save_path} ...")

    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(save_path, "wb") as f:
            for chunk in r.iter_content(chunk_size):
                if chunk:
                    f.write(chunk)

    return save_path

def pdf_to_images(pdf_path, out_dir="pdf_pages", dpi=200):
    os.makedirs(out_dir, exist_ok=True)
    doc = fitz.open(pdf_path)

    image_paths = []
    for i, page in enumerate(doc):
        pix = page.get_pixmap(dpi=dpi)
        img_path = f"{out_dir}/page_{i+1:03d}.png"
        pix.save(img_path)
        image_paths.append(img_path)

    return image_paths

def ocr_caption(crop):
    text = pytesseract.image_to_string(crop, lang="vie")
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    return text

def caption_to_filename(text, max_len=60):
    text = text.lower().strip()
    text = re.sub(r"(?<!\d)\.(?!\d)", "", text)
    text = re.sub(r"[^a-z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹ.\s]", "", text)
    text = re.sub(r"\s+", "_", text)
    return text[:max_len] if text else "figure"

def match_figures_and_captions(figures, captions):
    pairs = []

    for fig in figures:
        fx1, fy1, fx2, fy2 = fig["bbox"]
        f_center_y = (fy1 + fy2) / 2

        best_cap = None
        min_dist = float("inf")

        for cap in captions:
            cx1, cy1, cx2, cy2 = cap["bbox"]
            c_center_y = (cy1 + cy2) / 2

            if c_center_y < f_center_y:
                continue

            dist = abs(c_center_y - f_center_y)
            if dist < min_dist:
                min_dist = dist
                best_cap = cap

        pairs.append((fig, best_cap))

    return pairs


In [6]:
def detect_and_crop_diagrams(
    image_path,
    save_dir="cropped_diagrams",
    conf=0.25
):
    os.makedirs(save_dir, exist_ok=True)

    img = cv2.imread(image_path)

    results = model.predict(
        source=image_path,
        imgsz=1280,
        conf=conf,
        device=DEVICE,
        verbose=False
    )[0]

    names = model.names
    boxes = results.boxes.xyxy.cpu().numpy()
    classes = results.boxes.cls.cpu().numpy()
    scores = results.boxes.conf.cpu().numpy()

    figures = []
    captions = []

    for box, cls, score in zip(boxes, classes, scores):
        class_name = names[int(cls)]
        x1, y1, x2, y2 = map(int, box)

        if class_name == "figure":
            figures.append({
                "bbox": [x1, y1, x2, y2]
            })

        elif class_name == "figure_caption":
            captions.append({
                "bbox": [x1, y1, x2, y2],
                "crop": img[y1:y2, x1:x2]
            })

    pairs = match_figures_and_captions(figures, captions)

    outputs = []

    for idx, (fig, cap) in enumerate(pairs):
        fx1, fy1, fx2, fy2 = fig["bbox"]
        fig_crop = img[fy1:fy2, fx1:fx2]

        if cap is not None:
            caption_text = ocr_caption(cap["crop"])
            filename = caption_to_filename(caption_text)
        else:
            caption_text = ""
            filename = f"figure_{idx}"

        out_path = f"{save_dir}/{filename}.png"
        cv2.imwrite(out_path, fig_crop)

        outputs.append({
            "path": out_path,
            "caption": caption_text,
            "bbox": fig["bbox"]
        })

    return outputs

def extract_diagrams_from_pdf(pdf_path, output_root="output"):
    pages_dir = os.path.join(output_root, "pages")
    diagrams_dir = os.path.join(output_root, "diagrams")

    page_images = pdf_to_images(pdf_path, pages_dir)

    all_diagrams = []

    for page_img in page_images:
        diagrams = detect_and_crop_diagrams(
            page_img,
            save_dir=diagrams_dir
        )
        all_diagrams.extend(diagrams)

    print(f"Extracted {len(all_diagrams)} diagrams")
    return all_diagrams


In [ ]:
PDF_URL = "https://84864e12bc.vws.vegacdn.vn//data/doc/2025/thcslienninh/2025_2/26/sach-bai-tap-toan-8-tap-1-ket-noi-tri-thuc-voi-cuoc-song_262202515.pdf"
save_dir = "./input"

pdf_file_path = download_pdf_from_url(PDF_URL, save_dir=save_dir)
diagrams = extract_diagrams_from_pdf(pdf_file_path)

for d in diagrams[:5]:
    print(d)



PDF existed: ./input/sach-bai-tap-toan-8-tap-1-ket-noi-tri-thuc-voi-cuoc-song_262202515.pdf
